In [1]:
#export
from k1lib.callbacks import Callback, Callbacks
import k1lib, torch, numpy as np
import matplotlib.pyplot as plt

In [2]:
#export
@k1lib.patch(Callback.cls)
class LossLandscape(Callback):
    """Plots the loss landscape of the network. Display
    this in your cell for more."""
    def __init__(self): super().__init__()
    def _calc(self, i, ax, _range, vector1, vector2):
        res = 50; _range = k1lib.Range(_range); model = self.learner.model
        x, y = torch.meshgrid(torch.linspace(*_range, res), torch.linspace(*_range, res))
        z = np.empty((res, res))
        with torch.no_grad():
            originalParams = model.exportParams()
            for ix, _ in enumerate(x):
                for iy, _ in enumerate(y):
                    for param, og, v1, v2 in zip(model.parameters(), originalParams, vector1, vector2):
                        param.data = og + x[ix, iy] * v1 + y[ix, iy] * v2
                    self.cbs("startPass")
                    self.learner.y = self.learner.model(self.xb)
                    self.cbs("endPass")
                    self.cbs("startLoss"); self.learner.loss = self.lossF(self.y, self.yb).detach().item(); self.cbs("endLoss")
                    z[ix, iy] = self.loss
                    print(f"\rProgress: {round(100*(ix+iy/res)/res)}%     ", end="")
            model.importParams(originalParams)
        ax.plot_surface(x, y, z, cmap=plt.cm.coolwarm)
        print(f"     {i+1}/8 Finished {_range} range       ", end="")
    def plot(self):
        print() # nice new line
        self.cbs.suspend(["HookModule", "HookParam", "ProgressBar", "ParamScheduler", "Loss", "Autosave"])
        self.cbs("startRun"); self.cbs("startEpoch")
        self.learner.xb, self.learner.yb = next(iter(self.learner.data.valid))
        self.cbs("startBatch"); k1lib.clearLine()
        model = self.learner.model
        def getVectors(): return model.getParamsVector(), model.getParamsVector()
        fig, axes = plt.subplots(2, 4, subplot_kw={"projection": "3d"}, figsize=(16, 8), dpi=120)
        ranges = [.1, .32, 1, 3.2]
        for i, ax in enumerate(axes):
            vectors = getVectors()
            for j, _range in enumerate(ranges):
                self._calc(i*4+j, ax[j], [-_range, _range], *vectors)
        self.cbs("endBatch"); self.cbs("endEpoch"); self.cbs("endRun")
        plt.show(); self.cbs.restore()
    def __repr__(self):
        return f"""{self._reprHead}, use...
- ll.plot(): to plot everything
{self._reprCan}"""
@k1lib.patch(Callbacks, docs=LossLandscape)
def withLossLandscape(self): return self.append(LossLandscape())

In [3]:
!../../export.py _callbacks/lossLandscape

Current dir: /home/kelvin/repos/labs/k1lib, ../../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 0.1.0
Uninstalling k1lib-0.1.0:
  Successfully uninstalled k1lib-0.1.0
running install
running bdist_egg
running egg_info
creating k1lib.egg-info
writing k1lib.egg-info/PKG-INFO
writing dependency_links to k1lib.egg-info/dependency_links.txt
writing top-level names to k1lib.egg-info/top_level.txt
writing manifest file 'k1lib.egg-info/SOURCES.txt'
reading manifest file 'k1lib.egg-info/SOURCES.txt'
writing manifest file 'k1lib.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/k1lib
copying k1lib/_learner.py -> build/lib/k1lib
copying k1lib/callbacks.py -> build/lib/k1lib
copying k1lib/data.py -> build/lib/k1lib
copying k1lib/imports.py -> build/lib/k1lib
copying k1lib/_basics.py -> build/lib/k1lib
copying k1lib/nn.py -> 